In [ ]:
%matplotlib notebook
#%config InlineBackend.figure_format = 'svg'
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi, sqrt, meshgrid, mgrid
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, Button, interactive_output, HBox
import ipywidgets as widgets
from IPython.display import display, HTML
from scipy.integrate import odeint
import matplotlib.animation as animation
import time

In [ ]:
# Inicializamos algunas constantes que podemos usar más adelante
c=299792458         # Velocidad de la luz (m/s)
mu0=4*pi/1e7        # Permeabilidad magnética del vacío (NA^-2)
eps0= 1/(c**2*mu0)  # Permitividad del vacío (F/m)
ke=c**2/1e7         # Constante de Coulomb (Nm^2/C^2)

# 3. Componentes eléctricos

## Empecemos por la resistencia

¿Alguien ha visto alguna vez una resistencia en un circuito?

https://phet.colorado.edu/en/simulations/category/html (explorando un poco, se puede aprender mucho :D)

### $$\Delta V_R = I R$$

## Y de la bobina en un circuito, ¿qué me podéis decir?

Cuando llega la corriente –cambio de corriente– al primer bucle, se crea un campo magnético –que pasa por los demás bucles–. Como ha habido una variación del campo magnético, también se crea un campo eléctrico –en los demás bucles también–. Esto genera a su vez una diferencia de potencial y, por lo tanto, la bobina estaría actuando como una batería.
### Insisto de nuevo: ha de haber un cambio de corriente –como bien expresa la siguiente ecuación–.
### $$\displaystyle\Delta V_L = - L \dfrac{di}{dt}$$
L solo depende de la geometría de la bobina.

### Resumiendo:
+ Si no hay cambio en la corriente (**corriente continua**, DC): **no hace nada**, como si no estuviera.
+ Si sí hay cambio en la corriente (**corriente alterna**, AC): habrá una mayor diferencia de potencial cuanto mayor sea la frecuencia –variación– de la corriente. **Batería**.

## ¡Veamos ahora el condensador!

La carga eléctrica acumulada en las placas genera un campo eléctrico y, por consiguiente, también aparece un cambio de potencial entre las placas:
### $$\displaystyle\Delta V_C=\dfrac{Q}{C},$$
donde Q es la carga que se encuentra en cada placa por igual –como son de signo opuesto en cada placa, su suma neta es cero– y C solo depende de las propiedades físicas del condensador.

### A destacar –contrario a la bobina–:
+ En **corriente continua** (CC o, en inglés, DC), la corriente inyecta carga de manera constante al condensador, aumentando su diferencia de potencial hasta ser igual que la batería: en este momento, el condensador se comporta como **circuito abierto**.
+ En **corriente alterna** (AC): el condensador se carga y descarga constantemente, no se termina acumulando carga y, por lo tanto, no influye en nada, **como si no estuviera**.

https://phet.colorado.edu/sims/html/capacitor-lab-basics/latest/capacitor-lab-basics_en.html

¿Recordáis la sesión anterior, cuando jugábamos con las cargas?
Vamos a tomar prestado algunas funciones de allí. :)

In [ ]:
class carga:
    def __init__(self, q, posx, posy):
        self.q=q*1e-6
        self.pos=[posx,posy]
def E_field(cargas, X, Y):
    Ex, Ey = 0, 0
    for Q in cargas:
        r_x, r_y = X-Q.pos[0], Y-Q.pos[1]
        r = sqrt(r_x**2 + r_y**2)
        r[r==0.] = minVal; r_x[r_x==0.] = minVal; r_y[r_y==0.] = minVal
        Ex += ke*Q.q*np.divide(r_x,r**3)
        Ey += ke*Q.q*np.divide(r_y,r**3)
    return Ex, Ey
def Potencial(cargas, X, Y):
    V = 0
    for Q in cargas:
        r_x, r_y = X-Q.pos[0], Y-Q.pos[1]
        r = sqrt(r_x**2 + r_y**2)
        r[r==0.] = minVal # Se pone un valor mínimo en los ceros para evitar problemas numéricos
        V += ke*Q.q*np.divide(1,r)
    return V
def getCargas(q, posx, posy):
    return carga(q,posx,posy)

n=64                        # Nro. de puntos en cada eje del tablero (dominio del espacio vectorial)
xs = np.linspace(-2, 2, n)  # Eje x del tablero (dominio del espacio vectorial)
ys = np.linspace(-2, 2, n)  # Eje y del tablero (dominio del espacio vectorial)
X, Y = meshgrid(xs,ys)      # Se crea el tablero (se necesitan las coordenadas X e Y por separado)

n=1; d=0.4; q=1.0; Qs=[]; minVal=1e-6;
posy_placas=np.linspace(-.5,.5,n);
posx_placa1=-d/2*np.ones(n); posx_placa2=d/2*np.ones(n);
for i in xrange(n):
    Qs.append(carga(-q,posx_placa1[i],posy_placas[i]))
    Qs.append(carga(q,posx_placa2[i],posy_placas[i]))
    V = Potencial(Qs, X, Y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#line = plt.imshow(V, interpolation='bilinear', cmap=plt.cm.RdYlGn,origin='lower', extent=[-2, 2, -2, 2],vmax=abs(V).max(), vmin=-abs(V).max())
line = ax.plot_surface(X, Y, V*1e-6, rstride=1, cstride=1, cmap='Purples')
#ax.set_zlim([-100,100])
#ax.set_xlabel('$x$', fontsize=30)
#ax.set_ylabel('$y$', fontsize=30)
#ax.set_zlabel('$V (MV)$', fontsize=20)
#ax.set_title('Neuron potential with $Im='+str(I)+'$ $\mu A/cm^2$')

def init():
    line.set_data(([],[]))
    return line

def data(n):
    posy_placas=np.linspace(-.5,.5,n);
    posx_placa1=-d/2*np.ones(n); posx_placa2=d/2*np.ones(n);
    for i in xrange(n):
        Qs.append(carga(-q,posx_placa1[i],posy_placas[i]))
        Qs.append(carga(q,posx_placa2[i],posy_placas[i]))
    V = Potencial(Qs, X, Y)
    ax.clear()
    ax.grid(False)
    line = ax.plot_surface(X, Y, V*1e-6, rstride=1, cstride=1, cmap='Purples')
    #line = plt.imshow(V, interpolation='bilinear', cmap=plt.cm.RdYlGn,origin='lower', extent=[-2, 2, -2, 2],vmax=abs(V).max(), vmin=-abs(V).max())
    return line

#ani = animation.FuncAnimation(fig, data, init_func=init, interval=100, frames=xrange(1,20,1), blit=True)
ani = animation.FuncAnimation(fig, data, interval=100, frames=xrange(1,20,1), blit=False)
plt.show()


## ¡Perfecto, ya tenemos el condensador cargado!

### ¿Qué pasa si conectamos una resistencia y/o una bobina?
Vayamos por partes...

Lo suyo sería dibujar el circuito, pero dado que es muy sencillo, voy a dejar que lo hagáis vosotros, seríais capaces –clarto que sí–.
Para ello, poneros en mi piel, necesito poneros un circuito en la práctica. Lo primero, buscar en internet qué herramientas existen. ¡Qué bueno! Tras un rato buscando, he encontrado este [enlace](https://mybinder.org/v2/gh/psychemedia/showntell/electronics) donde podéis encontrar un resumen de las herramientas que existen para dibujar y/o analizar circuitos. :D
Siguiente paso, encontrar la herramienta que más os guste, aprender a manejarla y usarla para vuestro propio fin.

### Vayamos ahora a resolver el circuito. Para ello, apliquemos la ley de Kirchoff. 
### $$\Delta V_C + \Delta V_L \Delta V_R = 0$$
### $$\displaystyle\dfrac{Q}{C} + -L\dfrac{di}{dt} -IR = 0$$
Ya está casi... solo nos falta una ecuacioncilla más, muy sencillita:
$$i=\dfrac{dQ}{dt}$$
Pues vamos a sustituir:
### $$\displaystyle\dfrac{Q}{C} + -L\dfrac{d^2Q}{dQ^2} -IR = 0$$
## ¡¡¡¡¡Ecuación diferencial se segundo orden!!!!! :O

### XD XD XD, no os preocupéis, la podemos resolver muy fácilmente en el ordenador. :)


In [ ]:
%matplotlib notebook
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import numpy as np

# Damos unos valores razonables a las componentes
C=5e-3
L=300e-3
R=3.
V0=3.
dt=0.001  # Paso en el tiempo (1 ms)
T=2000    # 2 segundos
Q=np.zeros(T+1); dQ=np.zeros(T+1); ddQ=np.zeros(T+1); t=np.zeros(T+1)
Q[0]=V0*C; dQ[0]=0; ddQ[0]=0;
for i in xrange(T):
    #ddQ[i+1]=-Q[i]/(L*C)-dQ[i]*R/L # Circuito RLC
    ddQ[i+1]=-Q[i]/(L*C)          # Circuito LC
    dQ[i+1]=dQ[i]+ddQ[i+1]*dt
    Q[i+1]=Q[i]+dQ[i+1]*dt
    t[i+1]=t[i]+dt

In [ ]:
# Nuestra aproximación para la corriente en el circuito LC
I=np.divide(dQ,dt)
# Solución LC teórica
w=1/np.sqrt(L*C)
I_lc=np.divide(-Q[0]*w*np.sin(w*t),dt)
# Solución RLC teórica
w=np.sqrt(1/(L*C) - (R/(2*L))**2)
Q_rlc=Q[0]*np.exp(-R/(2*L)*t)*np.cos(w*t)
# Comparamos nuestra aproximación rápida numérica con la teórica
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(t,I,label='Numerica')
ax.plot(t,I_lc,label='Teorica') # Circuito RL
#ax.plot(t,Q,label='Numerica')
#ax.plot(t,Q_rlc,label='Teorica')  # Circuito RLC
ax.legend()
plt.show()

# Hodgkin–Huxley model

## Vamos a motivarnos un poco...

In [ ]:
from IPython.display import display, HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/aM9i9cU-MCY?rel=0&amp;controls=1&amp;showinfo=0;" frameborder="0" allowfullscreen></iframe>')

## ...para que la creatividad sea una de nuestras grandes fortalezas (o la mayor de todas)

Pues busquemos que hay que nos pueda interesar... y, si tras analizarlo, nos convence... aprovechémoslo (ganemos tiempo). :D

Buscando "Hodgkin–Huxley model python", uno de los primeros resultados que nos sale es esto:

https://www.bonaccorso.eu/2017/08/19/hodgkin-huxley-spiking-neuron-model-python/

In [ ]:
np.random.seed(1000) # Set random seed (for reproducibility)

#–––––––––––––– Constantes ––––––––––––––
gK = 36.0     # Average potassium channel conductance per unit area (mS/cm^2)
gNa = 120.0   # Average sodoum channel conductance per unit area (mS/cm^2)
gL = 0.3      # Average leak channel conductance per unit area (mS/cm^2)
Cm = 1.0      # Membrane capacitance per unit area (uF/cm^2)
VK = -12.0    # Potassium potential (mV)
VNa = 115.0   # Sodium potential (mV)
Vl = 10.613   # Leak potential (mV)
# Time values (in milliseconds)
tmin = 0.0; tmax = 200.0; dt=1e-3;
T = np.linspace(tmin, tmax, 1/dt)

#–––––––––––––– Ecuaciones auxiliares del modelo ––––––––––––––
# Potassium ion-channel rate functions
def alpha_n(Vm):
    return (0.01 * (10.0 - Vm)) / (np.exp(1.0 - (0.1 * Vm)) - 1.0)
def beta_n(Vm):
    return 0.125 * np.exp(-Vm / 80.0)

# Sodium ion-channel rate functions
def alpha_m(Vm):
    return (0.1 * (25.0 - Vm)) / (np.exp(2.5 - (0.1 * Vm)) - 1.0)
def beta_m(Vm):
    return 4.0 * np.exp(-Vm / 18.0)
def alpha_h(Vm):
    return 0.07 * np.exp(-Vm / 20.0)
def beta_h(Vm):
    return 1.0 / (np.exp(3.0 - (0.1 * Vm)) + 1.0)

# n, m, and h steady-state values
def n_inf(Vm=0.0):
    return alpha_n(Vm) / (alpha_n(Vm) + beta_n(Vm))
def m_inf(Vm=0.0):
    return alpha_m(Vm) / (alpha_m(Vm) + beta_m(Vm))
def h_inf(Vm=0.0):
    return alpha_h(Vm) / (alpha_h(Vm) + beta_h(Vm))
  
#–––––––––––––– Definición del sistema de ecuaciones diferenciales ordinarias (ODE) ––––––––––––––
def compute_derivatives(y, t0):
    dy = np.zeros((4,))
    Vm = y[0]
    n = y[1]
    m = y[2]
    h = y[3]
    # dVm/dt
    GK = (gK / Cm) * np.power(n, 4.0)
    GNa = (gNa / Cm) * np.power(m, 3.0) * h
    GL = gL / Cm
    dy[0] = (Id(t0) / Cm) - (GK * (Vm - VK)) - (GNa * (Vm - VNa)) - (GL * (Vm - Vl))
    # dn/dt
    dy[1] = (alpha_n(Vm) * (1.0 - n)) - (beta_n(Vm) * n)
    # dm/dt
    dy[2] = (alpha_m(Vm) * (1.0 - m)) - (beta_m(Vm) * m)
    # dh/dt
    dy[3] = (alpha_h(Vm) * (1.0 - h)) - (beta_h(Vm) * h)
    return dy

#–––––––––––––– Solve ODE system ––––––––––––––
# Input stimulus (dos estímulos de distinta duración y distinta amplitud...)
def Id(t):
    if 0.0 < t < 1.0:
        return 150.0
    elif 10.0 < t < 12.0:
        return 50.0
    return 0.0
Y = np.array([0.0, n_inf(), m_inf(), h_inf()]) # Inicialización
Vy = odeint(compute_derivatives, Y, T)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(3,1,1)
ax.plot(T, [Id(t) for t in T])
ax.set_xlabel('Time (ms)')
ax.set_ylabel(r'Current density ($\mu A/cm^2$)')
ax.set_title('Stimulus (Current density)')
plt.grid()
# Neuron potential
ax = fig.add_subplot(3,1,2)
ax.plot(T, Vy[:, 0])
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Vm (mV)')
ax.set_title('Neuron potential with two spikes')
plt.grid()
# n, m, h
ax = fig.add_subplot(3,1,3)
ax.plot(T, Vy[:, 1], label='n')
ax.plot(T, Vy[:, 2], label='m')
ax.plot(T, Vy[:, 3], label='h')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('')
ax.legend()
plt.grid()

Hemos aplicado dos impulsos de distinta duración e intensidad... y parecen prácticamente iguales...
¿Pero qué pasa si aumentamos su duración? ¿Y si variamos la intensidad? ¿Qué pasa? ¿Por qué?
Este puede ser un buen entregable para esta sesión. ;)

In [ ]:
I=1 #(uA/cm^2)
def Id(t):
    return I
plt.ion()
fig, ax = plt.subplots()
for I in plb.frange(0,7,0.4):
    plt.cla()
    Vy = odeint(compute_derivatives, Y, T)
    ax.plot(T, Vy[:, 0])
    ax.set_ylim([-70,120])
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Vm (mV)')
    ax.set_title('Neuron potential with $Im='+str(I)+'$ $\mu A/cm^2$')
    fig.canvas.draw()
    time.sleep(1e-6)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-2, 2))
line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

def data(i, z, line):
    z = np.sin(x+y+i)
    ax.clear()
    line = ax.plot_surface(x, y, z,color= 'b')
    return line,

n = 2.*np.pi
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = np.linspace(0,n,100)
y = np.linspace(0,n,100)
x,y = np.meshgrid(x,y)
z = np.sin(x+y)
line = ax.plot_surface(x, y, z,color= 'b')

ani = animation.FuncAnimation(fig, data, fargs=(z, line), interval=30, blit=False)

plt.show()